In [23]:
import numpy as np
import cma
import re
import math

x0 = 0.3
y0 = 0.3 
thetha0 = math.pi
delta_t = 0.01

input_seq = [
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],[1,1],
                [1,1],[1,1],[1,1],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0.3],[1,0.3],
                [1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[1,0.3],[0.3,1],
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]
            ]

In [24]:
def generate_path(L, r):
    x = x0
    y = y0
    theta = thetha0

    path = [[x,y,theta]]

    for wl, wr in input_seq:
        v = (wl + wr) * r / 2      
        w = (wl - wr) * r / L 

        x = x + v * delta_t * math.cos(theta)
        y = y + v * delta_t * math.sin(theta)
        theta = theta + w * delta_t

        if theta > math.pi:
            theta = theta - 2*math.pi
        elif theta < -math.pi:
            theta = (2*math.pi + theta)
        
        path.append([x,y,theta])
    
    return path

In [25]:
def loss(params):
    real = generate_path(real_params[0], real_params[1])
    pred = generate_path(params[0], params[1])
    return np.sqrt(np.mean((np.array(pred) - np.array(real)) ** 2))

In [26]:
def dist2target(path,target=[0,0]):
    return [np.linalg.norm(np.array(target)-np.array(step[0:-1])) for step in path]

In [32]:
L = 0.8
r = 0.69
real_params = [L,r]

L_fake = 0.6
r_fake = 0.13
fake_params = [L_fake,r_fake]

test_loss = loss(real_params)


In [33]:
es = cma.CMAEvolutionStrategy(len(real_params) * [0], 0.5, {'verbose': -3})
es.optimize(loss)

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 4.814786508091922e-02 1.0e+00 4.79e-01  4e-01  5e-01 0:00.0
    2     12 9.548683667499348e-03 1.4e+00 6.29e-01  6e-01  7e-01 0:00.0
    3     18 2.300800026275202e-02 1.8e+00 1.03e+00  9e-01  1e+00 0:00.0
  100    600 5.290247991001608e-11 3.9e+00 5.68e-06  9e-10  2e-09 0:00.3
  121    726 3.006860771946620e-13 3.7e+00 1.33e-07  3e-12  6e-12 0:00.4


In [30]:
print(f'Best loss value: {loss(es.result.xbest):.2}')

Best loss value: 5.6e-14


In [36]:
print('The parameters set is: L = {}, r = {}'.format(es.result.xbest[0],es.result.xbest[1]))


The parameters set is: L = 0.7999999999983491, r = 0.6899999999976001
